In [1]:
# installs
!pip install --upgrade pip
!pip install pandas==1.5.3
!pip install statsmodels
!pip install lifelines==0.26.4

In [2]:
# Imports here.
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
from lifelines import CoxPHFitter

import warnings
warnings.filterwarnings("ignore")

In [9]:
# Load one df
ndd = 'PD'
df = pd.read_csv(f'{ndd}_with_tenure_MAY_05_2025.csv', parse_dates = True)
df

,ID,date_of_birth,sex_at_birth,PD_DATE,DATE_OF_DEATH,recruit_date,felodipine_DATE,felodipine_N,chlorpromazine_DATE,chlorpromazine_N,...,QC0_irbesartan,QC10+_irbesartan,QC0_metformin,QC10+_metformin,QC0_zolpidem,QC10+_zolpidem,QC0_montelukast,QC10+_montelukast,QC0_esomeprazole,QC10+_esomeprazole
0,2525576,1951-06-15,Male,2017-12-05,NaN,2017-08-22,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,1163906,1934-06-15,Male,2018-01-25,NaN,2017-10-13,NaN,NaN,NaN,NaN,...,0,0,1,1,1,0,1,0,1,1
2,1857219,1944-06-15,Female,2018-03-06,NaN,2017-08-28,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,2429832,1950-06-15,Male,2018-03-17,NaN,2018-03-07,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,2277629,1942-06-15,Male,2018-04-05,NaN,2017-11-30,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65910,1289022,1954-06-15,Male,NaN,2020-03-24,2019-09-30,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
65911,2484058,1939-06-15,Female,NaN,2019-05-22,2019-05-14,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
65912,1839095,1940-06-15,Female,NaN,2020-12-30,2019-10-14,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
65913,3123992,1950-06-15,Female,NaN,2019-12-30,2018-01-17,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0


In [11]:
df.PD.value_counts()

0    65367
1      548
Name: PD, dtype: int64

In [4]:
#Select NDDs
ndd_list = ['AD', 'PD', 'DEM']

#Load list of codes
meds2 = pd.read_csv('meds_with_cleaned_name_MAY_05_2025.csv')
codes = list(set(list(meds2['cleaned_med'])))
print(len(codes))

187


In [5]:
print(ndd_list)

['AD', 'PD', 'DEM']


In [6]:
timeline = 'ever taken'
model = 'COX'
lag = "0"

results = []

for ndd in ndd_list:
    
    #Load df
    df = pd.read_csv(f'{ndd}_with_tenure_MAY_05_2025.csv', parse_dates = True)
    
    # Find codes to use so we don't have to use EVERYTHING
    codes_with_data = []
    lag = '0'

    for code in codes:
        m = df[['age_at_tenure', 'SEX', 'tenure', ndd, f'QC{lag}_' + code]]
        n=sum(m[f'QC{lag}_'+ code])
        df_pair = m[m[f'QC{lag}_'+ code]==1]
        n_pairs = sum(df_pair[ndd])
        if n == 0:
            pass
        elif n_pairs < 10:
            pass
        elif n == n_pairs:
            pass
        else:
            print(code)
            codes_with_data.append(code)
    
    print(ndd)
    print(len(codes_with_data))
    
    for code in codes_with_data:
        
        m = df[['age_at_tenure', 'SEX', 'tenure', ndd, f'QC{lag}_' + code]]
        n=sum(m[f'QC{lag}_'+ code])
        df_pair = m[m[f'QC{lag}_'+ code]==1]
        n_pairs = sum(df_pair[ndd])
        
        cph = CoxPHFitter()
        cph.fit(m, duration_col = 'tenure', event_col = ndd, show_progress=False, step_size = 0.01)
        #cph.print_summary()
        #cph.plot()
        
        actual_p = cph._compute_p_values()
        results_df = cph.summary
        results_df = results_df.reset_index()
        test = results_df.iloc[2]

        covariate = code
        HR = test['exp(coef)']
        ci_min = test['exp(coef) lower 95%']
        ci_max = test['exp(coef) upper 95%']
        p = actual_p[2]

        print(covariate, ndd, HR, ci_min, ci_max, p, n_pairs, n)
        results.append((covariate, ndd, model, timeline, lag, HR, ci_min, ci_max, p, n_pairs, n))
            
cox1 = pd.DataFrame(results, columns=('PRIOR','OUTCOME', 'MODEL','TIMELINE', 'LAG', 'HR', 'ci_min', "ci_max", 'P_VAL', "N_pairs", "N"))

lamotrigine
nitrofurantoin
oxybutynin
methotrexate
docusate
metformin
levothyroxine
losartan
aspirin
budesonide
bisacodyl
insulin
sildenafil
prednisolone
erythromycin
diclofenac
codeine
salmeterol
simvastatin
ibuprofen
acetaminophen
hydroxychloroquine
esomeprazole
vardenafil
pioglitazone
lansoprazole
nystatin
memantine
metoclopramide
loperamide
prochlorperazine
spironolactone
doxycycline
indomethacin
enalapril
nortriptyline
duloxetine
ciprofloxacin
atorvastatin
pramipexole
alendronate
paroxetine
diltiazem
hydroxyzine
amoxicillin
naproxen
meloxicam
clopidogrel
fenofibrate
propranolol
colchicine
valsartan
levetiracetam
lisinopril
baclofen
trospium
clarithromycin
celecoxib
escitalopram
ranitidine
clonidine
temazepam
gabapentin
diazepam
atropine
citalopram
clonazepam
rivaroxaban
morphine
allopurinol
oxycodone
ropinirole
pseudoephedrine
etodolac
sitagliptin
lidocaine
pregabalin
cimetidine
olanzapine
omeprazole
fluvastatin
tramadol
amlodipine
rosuvastatin
mirtazapine
pantoprazole
venlafaxine

morphine AD 1.4899581351589049 1.1007802769318962 2.0167287614506817 0.00983367019119055 48 5404
allopurinol AD 1.4124666804061001 0.9292293459714169 2.14700722906226 0.1060066765208426 24 2226
oxycodone AD 1.1098241306753076 0.8989361502267914 1.3701858588270768 0.33249886813212487 133 19826
ropinirole AD 5.121740685071957 2.881822491192068 9.102652132564343 2.5873018149326756e-08 12 477
pseudoephedrine AD 1.6454479419348145 1.069666701063429 2.5311612738115583 0.023423144461014136 22 2517
etodolac AD 1.5010245490340108 0.880176759706504 2.559797985979596 0.1358785665498996 14 1573
sitagliptin AD 1.684635643988318 0.9246720512182434 3.0691932877790684 0.08836641015497819 11 1007
lidocaine AD 1.3335280121793063 1.0332113384428714 1.7211357377735725 0.027040711434711855 73 9191
pregabalin AD 2.570587241555964 1.7889285354446067 3.693785769261049 3.316765033939381e-07 32 2278
cimetidine AD 2.411476736057695 1.2863844306174141 4.520592686088712 0.0060432034580213375 10 621
olanzapine AD 4

levothyroxine PD 1.439664297938137 1.1589928350259546 1.7883055253842828 0.0009893751372037425 106 10823
losartan PD 1.3728964934057295 1.102119201028091 1.710200475454476 0.004691987579862871 98 8744
aspirin PD 1.853132717562318 1.5618130393232472 2.1987912653027535 1.5575717280765981e-12 305 24050
budesonide PD 1.3578199830957525 0.9840362350090045 1.873584570264609 0.06260268532737218 40 3691
anastrozole PD 1.5855545746766584 0.8879571170797108 2.831199008287696 0.11917643607545726 12 1211
bisacodyl PD 1.8264861619449517 1.4695741839926366 2.2700805009467397 5.624834087062903e-08 100 7091
insulin PD 2.2374145405953683 1.7446152777506712 2.869414185643175 2.2331496534780496e-10 72 3763
sildenafil PD 0.9631844521420746 0.7324758653143869 1.2665595315554032 0.7883152508725664 63 5641
prednisolone PD 1.3099529820780702 1.024979661524089 1.6741569415179076 0.030998942538210332 77 6976
erythromycin PD 1.4037602200797545 0.8879103362812294 2.21930376858934 0.14670935288994771 19 1702
diclo

fluvastatin PD 1.472923101592653 1.039127779260199 2.0878110531795095 0.029585530263093917 34 2185
tramadol PD 1.789143046648362 1.4757695717581953 2.169060063731055 3.1946339095082647e-09 140 10888
amlodipine PD 1.4704674919129415 1.2089508221837424 1.7885546749263082 0.00011379151978590596 136 11112
rosuvastatin PD 1.3114906866944382 1.0313966639066434 1.667649199844547 0.02695565860724541 78 7144
mirtazapine PD 2.8583566834597964 2.0790317327950656 3.9298115565052996 1.0056867692675309e-10 41 1764
pantoprazole PD 2.047152510796762 1.6995720272042776 2.465816885298604 4.4686444331872306e-14 155 10386
venlafaxine PD 2.7661437384075604 2.0897616375215704 3.66144685793256 1.145537228200148e-12 56 3111
montelukast PD 1.3834639858298374 1.0158039087248893 1.884194954999472 0.03945095319449989 44 4278
metoprolol PD 1.4016650783996973 1.158452329666357 1.6959394372067673 0.0005153514325226134 150 12609
cetirizine PD 1.3159962902694033 1.0081059901649936 1.7179207869991786 0.0434524910619577

anastrozole DEM 0.8822549638882466 0.5876026815294766 1.3246600905213959 0.5457632745240522 24 1223
bisacodyl DEM 1.763557702579476 1.5390881270974628 2.020765228169712 3.147680078595839e-16 254 7245
valproate DEM 4.1916658925072685 2.3741495581512146 7.400571246274318 7.767714599796409e-07 12 144
insulin DEM 2.2674152551010316 1.9399975787149026 2.65009193592422 7.953465408306109e-25 183 3874
sildenafil DEM 0.931747506093733 0.77229762849697 1.1241176757223426 0.4603850796153365 134 5712
prednisolone DEM 1.286575577262795 1.1177876737032209 1.480850750952707 0.0004450114726666687 242 7141
erythromycin DEM 1.5297076927781839 1.1758925160996374 1.9899825820019743 0.0015390341829268788 58 1741
diclofenac DEM 1.3711320144076908 1.1265689645057555 1.6687864304503368 0.0016387575111815227 108 4097
lithium DEM 3.3856050846195043 2.327749490038469 4.924207625457157 1.767918594871542e-10 28 465
codeine DEM 1.3715490445510614 1.0447191184442706 1.8006244438315762 0.022907226679983562 54 1831
sa

morphine DEM 2.0076050800074037 1.7439121975911447 2.311170346098163 2.9919120043286546e-22 232 5588
allopurinol DEM 1.4865254914694532 1.193130400014299 1.852067499715049 0.000409165705033682 87 2289
oxycodone DEM 1.4654783658574342 1.3180723828264718 1.6293694252138178 1.5961412274502404e-12 584 20277
ropinirole DEM 7.922668511556705 6.268017603490691 10.014119345972471 3.5604928338833286e-67 74 539
pseudoephedrine DEM 1.6454669499935326 1.3212788326020457 2.0491976536011807 8.64696862206128e-06 85 2580
etodolac DEM 1.773157701255906 1.3745978594851194 2.287278575205119 1.0373959287032932e-05 62 1621
sitagliptin DEM 2.199887430879769 1.66753327117297 2.902193792594378 2.4429621632465813e-08 52 1048
lidocaine DEM 1.3287595964876395 1.163654398018266 1.5172907594083442 2.6821811429003663e-05 270 9388
pregabalin DEM 3.0996453434754168 2.6053727232845154 3.6876878188916353 2.60017193992123e-37 142 2388
cimetidine DEM 1.5310242221310841 1.01379906368111 2.312129940464619 0.042854706040107

In [7]:
#Combine results
output = pd.concat([cox1])

#Adding FDR Correction

#Sort P-values
output = output.sort_values(by = "P_VAL")

#Drop Nan-values
output = output.dropna()

#FDR Correction
rejected, p_corr = fdrcorrection(output['P_VAL'], is_sorted=True)
output['P_CORR'] = p_corr
output['SIGNIFICANT'] = rejected

output


,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
391,donepezil,DEM,COX,ever taken,0,18.241408,15.453108,21.532819,5.980110e-258,169,379,2.607328e-255,True
177,levodopa,PD,COX,ever taken,0,49.193106,38.713594,62.509351,6.358560e-223,79,253,1.386166e-220,True
96,donepezil,AD,COX,ever taken,0,42.720305,33.574459,54.357524,6.207195e-205,100,310,9.021124e-203,True
327,levodopa,DEM,COX,ever taken,0,16.364216,13.112633,20.422104,5.044588e-135,84,258,5.498601e-133,True
27,memantine,AD,COX,ever taken,0,38.194963,27.906755,52.276059,1.453376e-114,46,160,1.267344e-112,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,losartan,AD,COX,ever taken,0,1.058763,0.808715,1.386124,6.778394e-01,64,8710,6.841157e-01,False
142,sildenafil,PD,COX,ever taken,0,0.963184,0.732476,1.266560,7.883153e-01,63,5641,7.937770e-01,False
153,hydroxychloroquine,PD,COX,ever taken,0,1.079767,0.576594,2.022039,8.105147e-01,10,1334,8.130348e-01,False
12,sildenafil,AD,COX,ever taken,0,1.043841,0.734118,1.484235,8.111700e-01,39,5617,8.130348e-01,False


In [8]:
test = output[output['PRIOR']=='paroxetine']
test

,PRIOR,OUTCOME,MODEL,TIMELINE,LAG,HR,ci_min,ci_max,P_VAL,N_pairs,N,P_CORR,SIGNIFICANT
322,paroxetine,DEM,COX,ever taken,0,1.951103,1.517422,2.508731,1.875092e-07,64,1733,5.486847e-07,True
41,paroxetine,AD,COX,ever taken,0,2.443599,1.555549,3.838631,1.056074e-04,20,1689,2.083476e-04,True
173,paroxetine,PD,COX,ever taken,0,2.052367,1.382709,3.046346,3.596387e-04,26,1695,6.644172e-04,True


In [11]:
import os
import subprocess
import numpy as np
import pandas as pd

In [12]:
date = 'MAY_05_2025'
output.to_csv(f'AoU_{date}_results_lag_zero.csv', header = True, index = False)

In [13]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = output   

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = f'AoU_{date}_results_lag_zero.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr


b'Copying file://./AoU_MAY_05_2025_results_lag_zero.csv [Content-Type=text/csv]...\n/ [0 files][    0.0 B/ 61.5 KiB]                                                \r/ [1 files][ 61.5 KiB/ 61.5 KiB]                                                \r\nOperation completed over 1 objects/61.5 KiB.                                     \n'

In [ ]:
# This snippet assumes that you run setup first

# This code lists objects in your Google Bucket

# Get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# List objects in the bucket
print(subprocess.check_output(f"gsutil ls -r {my_bucket}", shell=True).decode('utf-8'))